# Bootcamp Ciência de Dados e IA – Manutenção Preditiva (IoT)

**Objetivo**: desenvolver um sistema que **prediz a classe do defeito** (entre 5 possíveis) e **retorna a probabilidade associada**, além de extrair **insights operacionais** e **visualizações** a partir de medidas de sensores de máquinas industriais.

**Arquivos**:
- `Bootcamp_train.csv`: treino/validação
- `Bootcamp_test.csv`: teste (sem rótulos) – gerar predições e enviar à API do desafio

> **Observação**: Este notebook foi gerado para ser executável de ponta a ponta. Se os arquivos CSV **não** estiverem presentes na pasta atual, ele **simula** um conjunto de dados sintético com o mesmo esquema para permitir a execução completa e a inspeção do pipeline.


## 1. Setup

In [3]:
# (Opcional) Caso deseje usar LightGBM/CatBoost/SHAP/Optuna, descomente:
# %pip install lightgbm catboost shap optuna

import warnings, os
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report, confusion_matrix, ConfusionMatrixDisplay,
    f1_score, log_loss, roc_auc_score, precision_recall_curve,
    average_precision_score
)
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.inspection import permutation_importance
plt.rcParams['figure.figsize']=(7,5)
plt.rcParams['axes.grid']=True
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 100)


In [2]:
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 3.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 5.0 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 3.7 MB/s eta 0:00:00 MB/s eta 0:00:01


## 2. Configurações do Projeto

In [4]:
TRAIN_PATH = Path('./dataset/bootcamp_train.csv')
TEST_PATH  = Path('./dataset/bootcamp_train.csv')
PROBLEM_KIND = 'multiclass'  # ou 'multilabel'
RANDOM_STATE = 42
TEST_SIZE = 0.20
OUT_DIR = Path('./outputs'); OUT_DIR.mkdir(parents=True, exist_ok=True)


## 3. Carregamento dos Dados (ou Simulação)

In [5]:
def simulate_data(n=6000, seed=7):
    rng=np.random.default_rng(seed)
    df=pd.DataFrame({'id':np.arange(n),'id_produto':rng.integers(1000,1400,size=n).astype(str),'tipo':rng.choice(list('LMH'),size=n,p=[0.45,0.35,0.20])})
    df['temperatura_ar']=rng.normal(300,5,n)
    df['temperatura_processo']=df['temperatura_ar']+rng.normal(15,8,n)
    df['umidade_relativa']=np.clip(rng.normal(45,12,n),5,95)
    df['velocidade_rotacional']=np.clip(rng.normal(1500,200,n),200,4000)
    df['torque']=np.clip(rng.normal(40,8,n),5,120)
    df['desgaste_da_ferramenta']=np.clip(rng.normal(80,50,n),0,500)
    delta_T=df['temperatura_processo']-df['temperatura_ar']
    potencia_rel=(df['velocidade_rotacional']*df['torque'])/(np.max(df['velocidade_rotacional'])*np.max(df['torque']))
    estresse_termico=delta_T*(1+(df['umidade_relativa']/100.0)*0.3)
    desgaste_risco=df['desgaste_da_ferramenta']/(df['desgaste_da_ferramenta'].max()+1e-9)
    tensao_excessiva=potencia_rel*(df['velocidade_rotacional']/df['velocidade_rotacional'].max())
    p_FDF=0.08+0.35*np.clip(desgaste_risco,0,1)
    p_FDC=0.05+0.25*np.clip(estresse_termico/(estresse_termico.max()+1e-9),0,1)
    p_FP=0.04+0.30*np.clip(potencia_rel,0,1)
    p_FTE=0.03+0.25*np.clip(tensao_excessiva,0,1)
    p_FA=0.03+0.05*rng.random(n)
    ps=np.vstack([p_FDF,p_FDC,p_FP,p_FTE,p_FA]).T
    max_p=ps.max(axis=1)
    no_fail_thresh=np.percentile(max_p,60)
    classes=np.where(max_p<no_fail_thresh,'sem_falha',np.array(['FDF','FDC','FP','FTE','FA'])[ps.argmax(axis=1)])
    df['classe_defeito']=classes
    for c in ['FDF','FDC','FP','FTE','FA']:
        df[c]=(df['classe_defeito']==c).astype(int)
    df['falha_maquina']=(df['classe_defeito']!='sem_falha').astype(int)
    return df
from pathlib import Path
if Path('Bootcamp_train.csv').exists():
    train=pd.read_csv('Bootcamp_train.csv'); print('Train carregado de Bootcamp_train.csv')
else:
    print('Train não encontrado. Simulando dados...'); train=simulate_data(6000,7)
if Path('Bootcamp_test.csv').exists():
    test=pd.read_csv('Bootcamp_test.csv'); print('Test carregado de Bootcamp_test.csv')
else:
    print('Test não encontrado. Simulando dados...'); test=simulate_data(1500,17).drop(columns=['classe_defeito','FDF','FDC','FP','FTE','FA','falha_maquina'],errors='ignore')
print(train.shape,test.shape); train.head()


Train não encontrado. Simulando dados...
Test não encontrado. Simulando dados...
(6000, 16) (1500, 9)


,id,id_produto,tipo,temperatura_ar,temperatura_processo,umidade_relativa,velocidade_rotacional,torque,desgaste_da_ferramenta,classe_defeito,FDF,FDC,FP,FTE,FA,falha_maquina
0,0,1377,L,299.204370,323.710407,50.578200,1354.886565,38.056956,104.681018,FDF,1,0,0,0,0,1
1,1,1250,H,300.743964,310.018732,32.274149,1475.727050,42.721579,64.209308,sem_falha,0,0,0,0,0,0
2,2,1273,M,297.769044,323.096653,34.448053,1732.120412,26.454887,28.565750,sem_falha,0,0,0,0,0,0
3,3,1358,H,299.711082,320.233333,55.149489,1755.263409,43.320499,26.848181,sem_falha,0,0,0,0,0,0
4,4,1231,H,304.946226,318.255694,36.994232,1653.436067,27.869880,22.598292,sem_falha,0,0,0,0,0,0


## 4. Qualidade dos Dados e Consistência dos Rótulos

In [6]:
def data_quality_report(df):
    print('Dimensões:',df.shape)
    print('\nTipos:'); print(df.dtypes)
    print('\nValores ausentes (%):'); na=df.isna().mean().sort_values(ascending=False)*100; print(na[na>0].round(2))
    print('\nDuplicatas:',df.duplicated().sum())
data_quality_report(train)
label_cols=['FDF','FDC','FP','FTE','FA']
if set(label_cols).issubset(train.columns):
    train['sum_fail_bins']=train[label_cols].sum(axis=1)
    print('\nOcorrências de múltiplas falhas simultâneas:'); print(train['sum_fail_bins'].value_counts().sort_index())
    if 'falha_maquina' in train.columns:
        inco=((train['falha_maquina']==0)&(train['sum_fail_bins']>0)).sum(); print('Inconsistências falha_maquina=0 porém alguma falha binária=1:',inco)
    train.drop(columns=['sum_fail_bins'],inplace=True)


Dimensões: (6000, 16)

Tipos:
id                          int64
id_produto                 object
tipo                       object
temperatura_ar            float64
temperatura_processo      float64
umidade_relativa          float64
velocidade_rotacional     float64
torque                    float64
desgaste_da_ferramenta    float64
classe_defeito             object
FDF                         int64
FDC                         int64
FP                          int64
FTE                         int64
FA                          int64
falha_maquina               int64
dtype: object

Valores ausentes (%):
Series([], dtype: float64)

Duplicatas: 0

Ocorrências de múltiplas falhas simultâneas:
sum_fail_bins
0    3600
1    2400
Name: count, dtype: int64
Inconsistências falha_maquina=0 porém alguma falha binária=1: 0


## 5. Definição do Problema e Construção do Alvo

In [7]:
def build_targets(df,kind='multiclass'):
    label_cols=['FDF','FDC','FP','FTE','FA']
    if kind=='multiclass':
        if 'classe_defeito' in df.columns:
            y=df['classe_defeito'].astype(str)
        elif set(label_cols).issubset(df.columns):
            def to_class(row):
                for c in label_cols:
                    if row[c]==1: return c
                return 'sem_falha'
            y=df[label_cols].apply(to_class,axis=1)
        else:
            raise ValueError('Faltam rótulos.')
        classes=['sem_falha','FDF','FDC','FP','FTE','FA']
        y=pd.Categorical(y,categories=classes).astype(str)
        return y,classes
    else:
        assert set(label_cols).issubset(df.columns)
        return df[label_cols].astype(int).copy(),label_cols
y,classes=build_targets(train,PROBLEM_KIND)
print('Classes:',classes)
print('Distribuição:'); print(y.value_counts() if PROBLEM_KIND=='multiclass' else y.sum().sort_values(ascending=False))


Classes: ['sem_falha', 'FDF', 'FDC', 'FP', 'FTE', 'FA']
Distribuição:


AttributeError: 'numpy.ndarray' object has no attribute 'value_counts'

## 6. Seleção de Atributos (Features)

In [ ]:
CAT_COLS=['tipo']
NUM_COLS=['temperatura_ar','temperatura_processo','umidade_relativa','velocidade_rotacional','torque','desgaste_da_ferramenta']
X=train[CAT_COLS+NUM_COLS].copy(); X_test=test[CAT_COLS+NUM_COLS].copy()
print('X:',X.shape,'X_test:',X_test.shape); X.head()


## 7. Análise de Balanceamento do Alvo

In [ ]:
import matplotlib.pyplot as plt
if PROBLEM_KIND=='multiclass':
    counts=y.value_counts().reindex(classes,fill_value=0)
    fig,ax=plt.subplots(); ax.bar(counts.index.astype(str),counts.values)
    ax.set_title('Distribuição de classes'); ax.set_xlabel('Classe'); ax.set_ylabel('Contagem'); plt.show()
else:
    sums=y.sum().reindex(classes,fill_value=0)
    fig,ax=plt.subplots(); ax.bar(sums.index.astype(str),sums.values)
    ax.set_title('Positivos por classe'); ax.set_xlabel('Classe'); ax.set_ylabel('N positivos'); plt.show()


## 8. Divisão Treino/Validação e Pesos de Classe

In [ ]:
from sklearn.model_selection import train_test_split
if PROBLEM_KIND=='multiclass':
    X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.20,stratify=y,random_state=42)
    vc=pd.Series(y_train).value_counts().reindex(classes,fill_value=1)
    inv=1.0/vc; class_weight=(inv/inv.mean()).to_dict(); sample_weight=pd.Series(y_train).map(class_weight).values
else:
    X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.20,random_state=42); sample_weight=None
print('Shapes:',X_train.shape,X_val.shape)


## 9. Modelo e Pipeline

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
pre=ColumnTransformer([('cat',OneHotEncoder(handle_unknown='ignore'),['tipo']),('num','passthrough',NUM_COLS)],remainder='drop')
if PROBLEM_KIND=='multiclass':
    base_model=HistGradientBoostingClassifier(learning_rate=0.10,max_iter=220,min_samples_leaf=25,random_state=42)
    pipe_base=Pipeline([('pre',pre),('clf',base_model)])
    clf=CalibratedClassifierCV(base_estimator=pipe_base,method='isotonic',cv=3)
    clf.fit(X_train,y_train,**({'clf__sample_weight':sample_weight} if sample_weight is not None else {}))
else:
    from sklearn.multiclass import OneVsRestClassifier
    base_model=HistGradientBoostingClassifier(learning_rate=0.10,max_iter=220,min_samples_leaf=25,random_state=42)
    clf=Pipeline([('pre',pre),('ovr',OneVsRestClassifier(base_model))])
    clf.fit(X_train,y_train)
print('Modelo treinado.')


## 10. Avaliação no Conjunto de Validação

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
if PROBLEM_KIND=='multiclass':
    proba_val=clf.predict_proba(X_val); y_pred=clf.predict(X_val); labels=np.array(['sem_falha','FDF','FDC','FP','FTE','FA'])
    macro_f1=f1_score(y_val,y_pred,average='macro'); ll=log_loss(y_val,proba_val,labels=labels); y_val_idx=pd.Categorical(y_val,categories=labels).codes
    ovr_auc=roc_auc_score(y_val_idx,proba_val,multi_class='ovr'); print(f'Macro-F1: {macro_f1:.4f} | LogLoss: {ll:.4f} | ROC-AUC OvR: {ovr_auc:.4f}')
    print('\nClassification report:'); print(classification_report(y_val,y_pred,zero_division=0))
    cm=confusion_matrix(y_val,y_pred,labels=labels,normalize='true'); fig,ax=plt.subplots(figsize=(7,7)); disp=ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=labels)
    disp.plot(ax=ax,values_format='.2f',colorbar=False); ax.set_title('Matriz de confusão normalizada (val)'); plt.show()
    fig,ax=plt.subplots(figsize=(7,6))
    for i,cls in enumerate(labels):
        y_true_cls=(pd.Series(y_val).values==cls).astype(int)
        precision,recall,_=precision_recall_curve(y_true_cls,proba_val[:,i]); ap=average_precision_score(y_true_cls,proba_val[:,i])
        ax.plot(recall,precision,label=f'{cls} (AP={ap:.2f})')
    ax.set_xlabel('Recall'); ax.set_ylabel('Precision'); ax.set_title('Precision–Recall (OvR)'); ax.legend(loc='best'); plt.show()
    top1_conf=proba_val.max(axis=1); top1_correct=(y_pred==pd.Series(y_val).values).astype(int)
    prob_true,prob_pred=calibration_curve(top1_correct,top1_conf,n_bins=10,strategy='uniform'); fig,ax=plt.subplots(figsize=(6,6))
    ax.plot(prob_pred,prob_true,marker='o'); ax.plot([0,1],[0,1],'--'); ax.set_xlabel('Probabilidade prevista (Top-1)'); ax.set_ylabel('Fração de acertos'); ax.set_title('Reliability diagram (Top-1)'); plt.show()
    pipe_base=Pipeline([('pre',pre),('clf',HistGradientBoostingClassifier(learning_rate=0.10,max_iter=220,min_samples_leaf=25,random_state=42))])
    pipe_base.fit(X_train,y_train,**({'clf__sample_weight':sample_weight} if sample_weight is not None else {}))
    perm=permutation_importance(pipe_base,X_val,y_val,n_repeats=8,random_state=42)
    ohe=pipe_base.named_steps['pre'].named_transformers_['cat']; feat_names=list(ohe.get_feature_names_out(['tipo']))+['temperatura_ar','temperatura_processo','umidade_relativa','velocidade_rotacional','torque','desgaste_da_ferramenta']
    imp_df=pd.DataFrame({'feature':feat_names,'importance_mean':perm.importances_mean,'importance_std':perm.importances_std}).sort_values('importance_mean',ascending=False)
    display(imp_df.head(15)); fig,ax=plt.subplots(figsize=(8,6)); ax.barh(imp_df['feature'],imp_df['importance_mean']); ax.invert_yaxis(); ax.set_xlabel('Queda média de score'); ax.set_title('Importâncias por permutação (val)'); plt.show()
else:
    Y_proba=clf.predict_proba(X_val); Y_pred=clf.predict(X_val); f1_macro=f1_score(y_val,Y_pred,average='macro'); print(f'F1 macro: {f1_macro:.4f}')
    aps=[]
    for i,c in enumerate(classes):
        ap=average_precision_score(y_val[c].values,Y_proba[:,i]); aps.append(ap)
    print('AP por classe:',{c:round(a,3) for c,a in zip(classes,aps)}); print('AP médio:',np.mean(aps).round(4))
    fig,ax=plt.subplots(figsize=(7,6))
    for i,c in enumerate(classes):
        precision,recall,_=precision_recall_curve(y_val[c].values,Y_proba[:,i]); ap=average_precision_score(y_val[c].values,Y_proba[:,i])
        ax.plot(recall,precision,label=f'{c} (AP={ap:.2f})')
    ax.set_xlabel('Recall'); ax.set_ylabel('Precision'); ax.set_title('Precision–Recall (multirrótulo)'); ax.legend(loc='best'); plt.show()


## 11. Predição no Conjunto de Teste e Arquivo de Submissão

In [ ]:
if PROBLEM_KIND=='multiclass':
    proba_test=clf.predict_proba(X_test); pred_test=clf.predict(X_test); labels=np.array(['sem_falha','FDF','FDC','FP','FTE','FA'])
    sub=pd.DataFrame({'id':test['id'] if 'id' in test.columns else np.arange(len(test)),'pred_class':pred_test})
    for i,cls in enumerate(labels): sub[f'proba_{cls}']=proba_test[:,i]
    sub_path=OUT_DIR/'submission_multiclass.csv'; sub.to_csv(sub_path,index=False); print('Arquivo de submissão salvo em:',sub_path)
else:
    Y_proba_test=clf.predict_proba(X_test)
    if isinstance(Y_proba_test,list): Y_proba_test=np.vstack([p[:,1] for p in Y_proba_test]).T
    sub=pd.DataFrame({'id':test['id'] if 'id' in test.columns else np.arange(len(test))})
    for i,cls in enumerate(classes): sub[f'proba_{cls}']=Y_proba_test[:,i]
    sub_path=OUT_DIR/'submission_multilabel.csv'; sub.to_csv(sub_path,index=False); print('Arquivo de submissão salvo em:',sub_path)
sub.head()


## 12. (Opcional) Validação Cruzada Estratificada

In [ ]:
if PROBLEM_KIND=='multiclass':
    from sklearn.model_selection import StratifiedKFold
    skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42); scores=[]
    for fold,(tr,va) in enumerate(skf.split(X,y),1):
        X_tr,X_va=X.iloc[tr],X.iloc[va]; y_tr,y_va=pd.Series(y).iloc[tr],pd.Series(y).iloc[va]
        vc=y_tr.value_counts().reindex(classes,fill_value=1); inv=1.0/vc; cw=(inv/inv.mean()).to_dict(); sw=y_tr.map(cw).values
        pipe=Pipeline([('pre',pre),('clf',HistGradientBoostingClassifier(learning_rate=0.10,max_iter=220,min_samples_leaf=25,random_state=42))])
        cal=CalibratedClassifierCV(base_estimator=pipe,method='isotonic',cv=3); cal.fit(X_tr,y_tr,clf__sample_weight=sw)
        proba_va=cal.predict_proba(X_va); y_hat=cal.predict(X_va); f1m=f1_score(y_va,y_hat,average='macro'); ll=log_loss(y_va,proba_va,labels=np.array(classes))
        scores.append((f1m,ll)); print(f'Fold {fold}: F1m={f1m:.4f} | LogLoss={ll:.4f}')
    print('\nMédia (F1m, LogLoss):',np.mean(scores,axis=0).round(4))
else:
    print('Para multirrótulo, use KFold simples ou estratificação multirrótulo.')


## 13. Próximos Passos e Extensões
- **Engenharia de atributos** (`delta_T`, razões, interações físicas)
- **Modelos**: LightGBM / CatBoost; ensembles (stacking)
- **Calibração por classe (multirrótulo)** com `CalibratedClassifierCV` por rótulo
- **Explainability**: SHAP (global/local) para insights operacionais
- **Tuning**: GridSearchCV/Optuna, early stopping
- **MLOps**: MLflow, testes de dados, CI/CD
- **API**: FastAPI para servir o modelo
- **Dashboard**: Streamlit para operação e interpretação
